In [7]:
import pandas as pd
import re
from urllib.parse import urlparse

In [25]:
# Read the Sales Data file
df1 = pd.read_csv('/Users/daan/Downloads/salestry.txt', delimiter='\t', encoding='ISO-8859-1') # If the delimiter is different, breplace '\t' with ',' or ' '

# View the columns to check if it works
print(df1.columns)
# Function to clean the title text
def clean_text(text):
    if pd.isna(text):  # Check if the text is NaN
        return ''  # Return an empty string or handle NaN as needed
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters using regex (keeping only alphanumeric characters and spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    return text

# Apply the cleaning function to the 'title' column
df1['title'] = df1['title'].apply(clean_text)
# Remove duplicates based on the 'title' column
df1 = df1.drop_duplicates(subset='title')

# Remove leading/trailing whitespace and ensure consistent casing
df1['title'] = df1['title'].str.strip().str.lower()
movies_df['TITLE'] = movies_df['TITLE'].str.strip().str.lower()
def clean_text(text):
    if pd.isna(text):  
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Count non-null values in the 'title' column
non_null_count = df1['title'].count()
print(f"Number of non-null rows in 'title' column: {non_null_count}")
# Count total rows in the DataFrame
total_rows = df1.shape[0]
print(f"Total number of rows in the DataFrame: {total_rows}")

# search_title = '808'
# result_df1 = df1.loc[df1['title'].str.contains(search_title, case=False, na=False)]

#print(df1)
df1.to_csv('/Users/daan/Desktop/HVA/Database management/Sales_correct_titles1.csv', index=False)


Index(['year', 'release_date', 'title', 'genre', 'international_box_office',
       'domestic_box_office', 'worldwide_box_office', 'production_budget',
       'Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
       'runtime', 'keywords', 'creative_type', 'url'],
      dtype='object')
Number of non-null rows in 'title' column: 29964
Total number of rows in the DataFrame: 29964


In [34]:
import pandas as pd
import re

# Load the CSV files
df1 = pd.read_csv('/Users/daan/Desktop/HVA/Database management/Sales_correct_titles.csv', encoding='ISO-8859-1')
movies_df = pd.read_csv('/Users/daan/Desktop/HVA/Database management/allMoviesWithUniqueID kopie.csv', encoding='ISO-8859-1')

# Function to clean the title text
def clean_text(text, remove_dots=False):
    if pd.isna(text):  
        return ''
    text = text.lower()
    if remove_dots:
        text = text.replace('.', '')  # Remove dots
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

# Apply the cleaning function to create 'clean_title' in both dataframes
df1['clean_title'] = df1['title'].apply(clean_text)
movies_df['clean_title'] = movies_df['TITLE'].apply(lambda x: clean_text(x, remove_dots=True))

# Print column names to ensure 'clean_title' exists
print("df1 columns:", df1.columns)
print("movies_df columns:", movies_df.columns)

# Check the first few rows to ensure 'clean_title' is created correctly
print("df1 head:\n", df1.head())
print("movies_df head:\n", movies_df.head())

# Merge dataframes on the 'clean_title' column
merged_df = pd.merge(df1, movies_df[['clean_title', 'ID']], on='clean_title', how='left')

# Rename 'ID' to 'Movie_ID'
merged_df.rename(columns={'ID': 'Movie_ID'}, inplace=True)

# Drop the temporary 'clean_title' column if no longer needed
merged_df.drop(columns=['clean_title'], inplace=True)

# Save the result to a new CSV file
merged_df.to_csv('/Users/daan/Desktop/HVA/Database management/Sales_correct_titles_with_Movie_ID5.csv', index=False)

# Check for unmatched titles
unmatched_titles = df1[~df1['clean_title'].isin(movies_df['clean_title'])]
print(f"Number of unmatched titles: {unmatched_titles.shape[0]}")


df1 columns: Index(['year', 'release_date', 'title', 'genre', 'international_box_office',
       'domestic_box_office', 'worldwide_box_office', 'production_budget',
       'Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
       'runtime', 'keywords', 'creative_type', 'url', 'clean_title'],
      dtype='object')
movies_df columns: Index(['TITLE', 'ID', 'clean_title'], dtype='object')
df1 head:
    year     release_date                title              genre  \
0  2000      January 1st         bakha satang              Drama   
1  2001     January 12th            antitrust  Thriller/Suspense   
2  2000     January 28th             santitos                NaN   
3  2002  2002 (Wide) by   frank mcklusky c i                 NaN   
4  2002     January 25th   a walk to remember              Drama   

  international_box_office domestic_box_office worldwide_box_office  \
0                 76576,00                 NaN             76576,00   
1               6900000,00  